# Suo

In [1]:
import warnings
import os
import sys
import gc
import warnings

In [2]:
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection
import scvi

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
tardis.config = tardis.config_server

In [3]:
torch.cuda.is_available()

False

In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [5]:
adata_file_path = os.path.join(tardis.config.io_directories["processed"], "dataset_complete_Suo.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path)

In [6]:
!ls /home/icb/kemal.inecik/lustre_workspace/tardis_data/models/suo_scanvi*

/home/icb/kemal.inecik/lustre_workspace/tardis_data/models/suo_scanvi:
model.pt

/home/icb/kemal.inecik/lustre_workspace/tardis_data/models/suo_scanvi_v2:
model.pt


In [7]:
model_name = "suo_scanvi_v2"
dir_path = os.path.join(
    tardis.config.io_directories["models"],
    model_name
)
vae = scvi.model.SCANVI.load(dir_path=dir_path, adata=adata)

/home/icb/kemal.inecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/kemal.inecik/tools/apps/mamba/envs/tardis_ ...
  rank_zero_warn(


INFO     File /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/models/suo_scanvi_v2/model.pt already         
         downloaded                                                                                                


In [8]:
biocons = BioConservation()
batchcor = BatchCorrection(
    silhouette_batch=False,
    ilisi_knn=False,
    kbet_per_label=False,
    graph_connectivity=False,
    pcr_comparison=False
)

In [9]:
def get_results(self, min_max_scale: bool = True) -> pd.DataFrame:
    _LABELS = "labels"
    _BATCH = "batch"
    _X_PRE = "X_pre"
    _METRIC_TYPE = "Metric Type"
    _AGGREGATE_SCORE = "Aggregate score"
    
    df = self._results.transpose()
    df.index.name = "Embedding"
    df = df.loc[df.index != _METRIC_TYPE]
    if min_max_scale:
        # Use sklearn to min max scale
        df = pd.DataFrame(
            MinMaxScaler().fit_transform(df),
            columns=df.columns,
            index=df.index,
        )
    df = df.transpose()
    df[_METRIC_TYPE] = self._results[_METRIC_TYPE].values

    # Compute scores
    per_class_score = df.groupby(_METRIC_TYPE).mean().transpose()
    # This is the default scIB weighting from the manuscript
    # per_class_score["Total"] = 0.4 * per_class_score["Batch correction"] + 0.6 * per_class_score["Bio conservation"]
    df = pd.concat([df.transpose(), per_class_score], axis=1)
    df.loc[_METRIC_TYPE, per_class_score.columns] = _AGGREGATE_SCORE
    return df

In [10]:
adata.obsm["scanvi"] = vae.get_latent_representation()

In [11]:
print(adata, flush=True)
print("starting scvi scib", flush=True)
bm = Benchmarker(
    adata=adata,
    batch_key="concatenated_integration_covariates",
    label_key="cell_type",
    embedding_obsm_keys=["scanvi"],
    pre_integrated_embedding_obsm_key="Unintegrated",  # equals to X_pca
    bio_conservation_metrics=biocons,
    batch_correction_metrics=batchcor,
    n_jobs=-1,
)

AnnData object with n_obs × n_vars = 841922 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes', '_scvi_batch', '_scvi_labels'
    uns: 'rank_genes_groups', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'Unintegrated', 'X_pca', 'harmony', 'scanvi'


starting scvi scib


In [12]:
bm.benchmark()

Computing neighbors:   0%|          | 0/1 [00:00<?, ?it/s]

Computing neighbors: 100%|██████████| 1/1 [05:19<00:00, 319.37s/it]

Computing neighbors: 100%|██████████| 1/1 [05:19<00:00, 319.39s/it]

Embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

...

Metrics:  30%|███       | 3/10 [57:51<2:16:45, 1172.16s/it, Bio conservation: silhouette_label]

Metrics:  30%|███       | 3/10 [57:51<2:16:45, 1172.16s/it, Bio conservation: clisi_knn]       

Metrics:  40%|████      | 4/10 [57:57<1:11:10, 711.77s/it, Bio conservation: clisi_knn] 

Embeddings: 100%|██████████| 1/1 [57:57<00:00, 3477.70s/it]

Embeddings: 100%|██████████| 1/1 [57:57<00:00, 3477.70s/it]

In [13]:
df = get_results(bm, min_max_scale=False)

In [14]:
df

,isolated_labels,nmi_ari_cluster_labels_kmeans_nmi,nmi_ari_cluster_labels_kmeans_ari,silhouette_label,clisi_knn,Bio conservation
Embedding,,,,,,
scanvi,0.774158,0.792213,0.360095,0.576235,0.999928,0.700526
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Aggregate score


In [15]:
pickle_path = os.path.join(tardis.config.io_directories["temp"], "latent", "suo_scib_scanvi_bio_2.pickle")
pickle_path

'/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/_temporary/latent/suo_scib_scanvi_bio_2.pickle'

In [16]:
df.to_pickle(pickle_path)